In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torchvision import datasets
import torchvision.transforms.functional as TF

# =========================
# 1. Hàm xử lý ảnh
# =========================
def extract_features(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None

    img = cv2.resize(img, (100, 100))

    # Làm mờ nền
    blurred = cv2.GaussianBlur(img, (7, 7), 0)
    enhanced = cv2.subtract(img, blurred)
    enhanced = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)

    # Binarize và invert
    _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = 255 - binary

    # Cắt vùng chữ
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        binary = binary[y:y+h, x:x+w]

    # Resize và flatten
    final = cv2.resize(binary, (28, 28), interpolation=cv2.INTER_AREA)
    return final.flatten() / 255.0

# =========================
# 2. Export ảnh MNIST ra thư mục
# =========================
def export_mnist_to_folder(out_dir, limit=2000):
    os.makedirs(out_dir, exist_ok=True)
    mnist = datasets.MNIST(root='./mnist_data', train=True, download=True)
    count = 0
    for idx, (img, label) in enumerate(mnist):
        if count >= limit:
            break
        img_np = np.array(img)
        filename = f"{label}_mnist_{idx:06d}.jpg"
        cv2.imwrite(os.path.join(out_dir, filename), img_np)
        count += 1
    print(f"✅ Đã lưu {count} ảnh MNIST vào {out_dir}")

# =========================
# 3. Load ảnh có nhãn và train SVM
# =========================
def load_dataset(data_dir):
    X, y = [], []
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    label = int(file.split('_')[0])
                    path = os.path.join(root, file)
                    feat = extract_features(path)
                    if feat is not None:
                        X.append(feat)
                        y.append(label)
                except:
                    continue
    return np.array(X), np.array(y)

# =========================
# 4. Train và đánh giá
# =========================
def train_svm_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = svm.SVC(kernel='rbf', C=10, gamma=0.01)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("\n--- Classification Report ---")
    print(classification_report(y_test, y_pred))
    return model

# =========================
# 5. Dự đoán tập không nhãn
# =========================
def predict_on_folder(model, input_dir, output_csv="svm_predictions.csv"):
    results = []
    for file in tqdm(os.listdir(input_dir)):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(input_dir, file)
            feat = extract_features(path)
            if feat is not None:
                pred = model.predict([feat])[0]
                results.append([file, pred])

    df = pd.DataFrame(results, columns=["filename", "predicted_digit"])
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Đã xuất kết quả vào {output_csv}")

# =========================
# 6. Main pipeline
# =========================
def main():
    labeled_dir = r"D:\\code_things\\do an cuoi ki mon may hoc\\hand_Written(CNN)\\merged_data"
    unlabeled_dir = r"D:\\code_things\\do an cuoi ki mon may hoc\\hand_Written(CNN)\\data no label\\data.2025"

    print("📦 Đang export ảnh MNIST...")
    export_mnist_to_folder(labeled_dir, limit=2000)

    print("🔄 Đang load dữ liệu có nhãn...")
    X, y = load_dataset(labeled_dir)

    print("🚀 Đang train SVM...")
    model = train_svm_model(X, y)

    print("🔍 Đang dự đoán dữ liệu không nhãn...")
    predict_on_folder(model, unlabeled_dir)

if __name__ == "__main__":
    main()


📦 Đang export ảnh MNIST...
✅ Đã lưu 2000 ảnh MNIST vào D:\\code_things\\do an cuoi ki mon may hoc\\hand_Written(CNN)\\merged_data
🔄 Đang load dữ liệu có nhãn...
🚀 Đang train SVM...

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.67      0.59      0.63       162
           1       0.48      0.63      0.55       166
           2       0.60      0.62      0.61       163
           3       0.49      0.43      0.46       151
           4       0.50      0.54      0.52       132
           5       0.57      0.49      0.53       145
           6       0.55      0.60      0.57       135
           7       0.51      0.48      0.50       156
           8       0.59      0.54      0.56       156
           9       0.52      0.54      0.53       155

    accuracy                           0.55      1521
   macro avg       0.55      0.54      0.54      1521
weighted avg       0.55      0.55      0.55      1521

🔍 Đang dự đoán dữ liệu không 

100%|██████████| 9998/9998 [03:12<00:00, 51.99it/s]


✅ Đã xuất kết quả vào svm_predictions.csv
